In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [5]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='2addf6c84ba0ca6544946a602bf943553d36b693'
)

detects_dict={}
for video_data in Ex.test_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.test_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

2017-06-19 15:28:04,187 [INFO] [holmes.experiment.staging] Preparing holmes payload
2017-06-19 15:28:04,189 [INFO] [holmes.experiment.staging] Cloning git:hudl/holmes
2017-06-19 15:28:14,656 [INFO] [holmes.experiment.staging] Checking out git:hudl/holmes:2addf6c84ba0ca6544946a602bf943553d36b693
2017-06-19 15:28:15,032 [INFO] [holmes.experiment.staging] Preparing distribution package
2017-06-19 15:28:17,860 [INFO] [holmes.experiment.staging] Staging distribution package
2017-06-19 15:28:17,862 [INFO] [holmes.experiment.staging] Commit staged
2017-06-19 15:28:17,863 [INFO] [holmes.experiment.staging] Removing Holmes Directory
2017-06-19 15:28:17,922 [INFO] [holmes.experiment.staging] Extracting default config from package


In [6]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
lost_thresholds = [1]
hungarian_thresholds = [14]
minimum_detections = [9]

In [7]:
configs = []
list_configs=[]
for lt in lost_thresholds:
    for ht in hungarian_thresholds:
        for min_det in minimum_detections:
            for config in list_dicts:
                current_config_dict = config.copy()
                parameter_map = {"tracker_config.tracklet_config.lost_threshold": lt,"tracker_config.associator_config._class_name": 'SpatialAssociatorConfig', "tracker_config.associator_config.threshold": ht}
                flat_dict = flatten_dict(current_config_dict)
                flat_dict.update(parameter_map)
                updated_config_dict = expand_dict(flat_dict)
                updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
                configs.append(updated_config_dict)
                file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}-{:02d}-{:02d}.json".format(len(list_configs), 
                                                    current_config_dict['video_config']['video_id'], lt, ht, min_det))
                list_configs.append(file_name)
                with open(file_name, 'w') as f:
                    json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

1 /var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpb5asu7xf/M00-AG0317-000-01-14-09.json


In [8]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-LT-HT-DET-2-5", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-LT-HT-DET-2-5",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

2017-06-19 15:28:30,146 [INFO] [holmes.experiment.base] Verifying AWS staging area: s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-5


In [9]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [10]:
experiment.stage_experiment()

2017-06-19 15:28:35,597 [INFO] [holmes.experiment.base] staging measurement M00-AG0317-000-01-14-09
2017-06-19 15:28:35,997 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpb5asu7xf/M00-AG0317-000-01-14-09.json, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-5/deps/M00-AG0317-000-01-14-09.json
2017-06-19 15:28:36,339 [INFO] [holmes.experiment.base] staging file, source=/var/folders/57/58_nn3s94l9f_jsjrzsd2h6mmwn1_p/T/tmpb5asu7xf/2addf6c84ba0ca6544946a602bf943553d36b693.tar.gz, destination=s3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-5/deps/2addf6c84ba0ca6544946a602bf943553d36b693.tar.gz


{'experiment_id': 'PTA-95-LT-HT-DET-2-5',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-5/deps/M00-AG0317-000-01-14-09.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-5',
   'job_id': 'PTA-95-LT-HT-DET-2-5-M00-AG0317-000-01-14-09',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {},
   'measurement_id': 'M00-AG0317-000-01-14-09',
   'package_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-5/deps/2addf6c84ba0ca6544946a602bf943553d36b693.tar.gz'}]}

In [11]:
experiment.launch_experiment()

2017-06-19 15:28:53,074 [INFO] [holmes.experiment.base] launching PTA-95-LT-HT-DET-2-5-M00-AG0317-000-01-14-09


{'experiment_id': 'PTA-95-LT-HT-DET-2-5',
 'job_queue': 'holmes-research-queue-cpu',
 'measurements': [{'config_path': 's3://hudl-holmes/experiments/PTA-95-LT-HT-DET-2-5/deps/M00-AG0317-000-01-14-09.json',
   'experiment_id': 'PTA-95-LT-HT-DET-2-5',
   'job_id': 'PTA-95-LT-HT-DET-2-5-M00-AG0317-000-01-14-09',
   'job_queue': 'holmes-research-queue-cpu',
   'launch_telemetry': {'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
      'content-length': '105',
      'content-type': 'application/json',
      'date': 'Mon, 19 Jun 2017 20:26:36 GMT',
      'via': '1.1 c157dc52e9f3c011b626279fe96418f8.cloudfront.net (CloudFront)',
      'x-amz-cf-id': 'pe3pcGQtGfVu693iKnL5Yeby2yXU4rTKtniVmmmIZqf8z-MZz977Fw==',
      'x-amzn-requestid': '97c04925-552d-11e7-af5a-6bb627506715',
      'x-amzn-trace-id': 'sampled=0;root=1-5948337c-f23b9fadf01d4085c80bc691',
      'x-cache': 'Miss from cloudfront'},
     'HTTPStatusCode': 200,
     'RequestId': '97c04925-552d-11e7-af5a-6bb627506715',


In [9]:
from holmes.io import Retrieve
import re

In [10]:
paths=[]
for m in experiment.measurements:
    measurement_path = m.__dict__['_s3_measurement_path']
    video_id = ''.join(re.match(r'.*/M\d{2}-(\w+)(-{1})(\d+)-.*', measurement_path).groups())
    path = m.__dict__['_s3_measurement_path']+'/trajectory_results_{}.json'.format(video_id)
    paths.append(path)

In [11]:
import pandas as pd
import numpy as np

In [14]:
data = pd.DataFrame({'game':[],'mota':[],'fnr':[],'fpr':[],'mr':[],'lt':[],'ht':[],'det':[],'job_id':[]})
for i,j in enumerate([Retrieve(j) for j in paths]):
    with open(j.local_path, 'r') as fp:
        results = json.load(fp)
        game = results['video_id'][0:6]
        mota = results['MOTA']
        fnr = results['False negative rate']
        fpr = results['False positive rate']
        mr = results['Mismatch rate']
        lt = configs[i]["tracker_config"]["tracklet_config"]["lost_threshold"]
        ht = configs[i]['tracker_config']['associator_config']['threshold']
        det = configs[i]['filter_configs'][0]['min_detections']
        job_id = paths[i][29:73].replace('/','-')
        row = pd.DataFrame({'game':[game],'mota':[mota],'fnr':[fnr],'fpr':[fpr],'mr':[mr],'lt':[lt],'ht':[ht],
                            'det':[det],'job_id':[job_id]})
        data = data.append(row)
data[['lt','ht','det']] = data[['lt','ht','det']].applymap(np.int64)

In [20]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [17]:
output_notebook()

Loading BokehJS ...

In [18]:
data_lt_ht_det = data.copy()
data_lt_ht_det['lost'] = data_lt_ht_det['lt']
data_lt_ht_det['hungarian'] = data_lt_ht_det['ht']
data_lt_ht_det['detections'] = data_lt_ht_det['det']
data_lt_ht_det = data_lt_ht_det.loc[:,['mota','ht', 'lt', 'det', 'lost', 'hungarian', 'detections','fpr','fnr','mr']].groupby(['ht','lt','det']).mean()
data_lt_ht_det

mota  lost  hungarian  detections       fpr       fnr        mr
ht lt det                                                                     
12 1  7    0.778357     1         12           7  0.053068  0.118218  0.013881
      9    0.779345     1         12           9  0.051355  0.123610  0.012886
      11   0.779020     1         12          11  0.049529  0.129985  0.011908
      13   0.777550     1         12          13  0.048260  0.136265  0.011140
14 1  7    0.778466     1         14           7  0.054125  0.116074  0.013570
      9    0.780031     1         14           9  0.052325  0.120767  0.012645
      11   0.779838     1         14          11  0.050858  0.126252  0.011842
      13   0.779038     1         14          13  0.049536  0.131797  0.011128
16 1  7    0.778380     1         16           7  0.054516  0.114997  0.013476
      9    0.779782     1         16           9  0.052942  0.119432  0.012618
      11   0.779955     1         16          11  0.051482  0.124359  0.011872
      13   0.779536     1         16          13  0.050182  0.129535  0.011159
18 1  7    0.778113     1         18           7  0.054851  0.114387  0.013471
      9    0.779537     1         18           9  0.053248  0.118573  0.012659
      11   0.779987     1         18          11  0.051769  0.123196  0.011932
      13   0.779308     1         18          13  0.050671  0.128185  0.011299

In [25]:
data[data['det']==9][data[data['det']==9]['ht']==14].groupby(['game']).mean()['mota']

game
ChvUtd    0.797572
ER0429    0.678067
PP0214    0.873521
PS0304    0.770965
Name: mota, dtype: float64

In [19]:
scat1 = Scatter(data_lt_ht_det, x='hungarian', y='mota', color='detections', marker='lost')
scat2 = Scatter(data_lt_ht_det, x='hungarian', y='mr', color='detections', marker='lost')
scat3 = Scatter(data_lt_ht_det, x='hungarian', y='fpr', color='detections', marker='lost')
scat4 = Scatter(data_lt_ht_det, x='hungarian', y='fnr', color='detections', marker='lost')
show(column(row(scat1, scat2), row(scat3, scat4)))

In [26]:
configs[0]

{'checkpoint_freq': 1500,
 'detector_config': {'_class_name': 'PassThroughDetectorConfig',
  'batch_size': 1,
  'source_path': 's3://hudl-holmes/recordings/stamford_bridge/chelsea_v_utd_u21/detections_000.json'},
 'filter_configs': [{'_class_name': 'BbAreaFilterConfig',
   'max_pixel_area': 5000,
   'min_detections': 7,
   'min_pixel_area': 100}],
 'output_location': '/otp/output/',
 'start_frame': 0,
 'tracker_config': {'_class_name': 'OnlineTrackerConfig',
  'associator_config': {'_class_name': 'SpatialAssociatorConfig',
   'threshold': 12},
  'tracklet_config': {'_class_name': 'SimpleTrackletConfig',
   'lost_threshold': 1,
   'motion_model_config': {'_class_name': 'DummyMotionModelConfig'}}},
 'video_config': {'_class_name': 'RawVideoConfig',
  'ground_truth_path': 's3://hudl-holmes/recordings/stamford_bridge/chelsea_v_utd_u21/truth_000.json',
  'video_id': 'ChvUtd-000',
  'video_path': 's3://hudl-holmes/recordings/stamford_bridge/chelsea_v_utd_u21/processed_000.mp4'}}